Loading the Segment Anything Model

Imports necessary libraries and sets up variables for loading a pre-trained Segment Anything Model (SAM).
Loads the specified SAM model from a checkpoint file and prepares it to run on the selected device (GPU or CPU).

In [ ]:
import torch
from segment_anything import sam_model_registry

CHECKPOINT_PATH = 'sam_vit_l_0b3195.pth'
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_l"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)

Load Image

Loads an image and prepares it for use with the Segment Anything Model (SAM).
Utilizes the SAM model to generate segmentation masks for the loaded image.

In [4]:
import cv2
from segment_anything import SamAutomaticMaskGenerator

mask_generator = SamAutomaticMaskGenerator(sam)

image_bgr = cv2.imread("dog_image.jpg")
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
result = mask_generator.generate(image_rgb)

Supervision
Imports tools for annotation and visualization from the 'supervision' library.
Applies the generated segmentations to the image, creating visually annotated output.

In [ ]:
import supervision as sv
from supervision.annotators.utils import ColorLookup

mask_annotator = sv.MaskAnnotator(color_lookup = ColorLookup.INDEX)
detections = sv.Detections.from_sam(result)
annotated_image = mask_annotator.annotate(image_bgr, detections)

Generate Segmentation Mask with Bounding Box

In [16]:
import cv2
from segment_anything import SamPredictor
import numpy as np

mask_predictor = SamPredictor(sam)

image_bgr = cv2.imread("dog_image.jpg")
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
mask_predictor.set_image(image_rgb)

box = np.array([70, 247, 626, 926])
masks, scores, logits = mask_predictor.predict(
    box=box,
    multimask_output=True
)

Export annotated image

In [7]:
cv2.imwrite("annotated_dog_image.jpg", annotated_image)  

True